# Essential Boundary Conditions

The following snippets illustrates different ways of defining essential boundary conditions. First we create a mesh of a simple beam with a rectangular cross section:

In [1]:
from neumann.linalg import linspace
from neumann.array import repeat, minmax
from polymesh.space import (StandardFrame, PointCloud)
from sigmaepsilon.solid.fem import (Structure, LineMesh, 
                                    PointData, NodalSupport)
from sigmaepsilon.solid.fem.cells import B2
import numpy as np

L = 100.  # length of the console
w, h = 10., 10.  # width and height of the rectangular cross section
F = -100.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
nElem = 20  # number of finite elements to use
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# load at the rightmost node
nodal_loads = np.zeros((coords.shape[0], 6))
nodal_loads[-1, 2] = F

One way to define the boundary conditions is through a boolean array called 'fixity' on the associated pointcloud:

In [2]:
# support at the leftmost node
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=nodal_loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = B2(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)

structure.linsolve()
dofsol = structure.nodal_dof_solution()
minmax(dofsol)

(-0.19047719057601065, 0.0028571528571387592)

Definition of the same condition using classes:

In [3]:
# support at the leftmost node
support = NodalSupport(x=[0, 0, 0], 
                       UX = 0., UY=0., UZ=0., 
                       UXX = 0., UYY=0., UZZ=0.)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=nodal_loads)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = B2(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
# add the nodal constraints
structure.constraints.append(support)

structure.linsolve()
dofsol = structure.nodal_dof_solution()
minmax(dofsol)

(-0.19047719057601065, 0.0028571528571387592)

The object oriented approach can also be used to define inclined supports, by passing an arbitrary frame. The following blocks define a simply supported beam with the support on the right side being inclined:

In [4]:
from numpy import pi
from polymesh.space import index_of_closest_point

# hinged support at the leftmost node
support_left = NodalSupport(x=[0, 0, 0], UX = 0., UY=0., 
                            UZ=0., UZZ=0., UXX=0.)

# inclined roller support at the rightmost node
angle = - 30 * pi / 180
frame = GlobalFrame.rotate_new('Body', [0, angle, 0], 'XYZ')
support_right = NodalSupport(x=[L, 0, 0], frame=frame, UZ=0.)

# concentrated load at the middle
x_middle = np.array([L/2, 0, 0], dtype=float)
i = index_of_closest_point(coords, x_middle)
nodal_loads = np.zeros((coords.shape[0], 6))
nodal_loads[i, 2] = F

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=nodal_loads)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = B2(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
# add the nodal constraints
structure.constraints.append(support_left)
structure.constraints.append(support_right)

structure.linsolve()
dofsol = structure.nodal_dof_solution()
minmax(dofsol)

(-0.011944444511127686, 0.00035793650827022115)

In [5]:
minmax(structure.internal_forces(flatten=False)[:, :, 0])  # -28.867

(-28.86751346795713, -28.86751346795692)